In [2]:
import requests
import pandas as pd
from io import BytesIO
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket


In [8]:
@task()
def extract_from_web(url: str) -> bytes:
    r = requests.get(url)
    return r.content

D:\DEZC\Week2\zoomcamp\lib\site-packages\prefect\tasks.py:256: UserWarning: A task named 'extract_from_web' and defined at 'C:\Users\ksale\AppData\Local\Temp\ipykernel_4392\1108753477.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [9]:
@task()
def save_as_raw(content: bytes, storage_path: str) -> None:
    gcs = GcsBucket.load('zoom-gcs')
    gcs.upload_from_file_object(BytesIO(content), storage_path)


D:\DEZC\Week2\zoomcamp\lib\site-packages\prefect\tasks.py:256: UserWarning: A task named 'save_as_raw' and defined at 'C:\Users\ksale\AppData\Local\Temp\ipykernel_4392\4108602559.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [10]:
@task(log_prints=True)
def save_as_parquet(content: bytes, url: str, storage_path_parquet: str) -> pd.DataFrame:
    gcs = GcsBucket.load('zoom-gcs')
    if url.endswith('.csv'):
        df = pd.read_csv(BytesIO(content))
    if url.endswith('.csv.gz'):
        df = pd.read_csv(BytesIO(content), compression='gzip')
    df.columns = [c.lower() for c in df.columns]
    print(df.head())
    print('Rows loaded:', len(df))
    buffer = BytesIO()
    df.to_parquet(buffer, engine='auto', compression='snappy')
    buffer.seek(0)
    gcs.upload_from_file_object(buffer, storage_path_parquet)


D:\DEZC\Week2\zoomcamp\lib\site-packages\prefect\tasks.py:256: UserWarning: A task named 'save_as_parquet' and defined at 'C:\Users\ksale\AppData\Local\Temp\ipykernel_4392\2229267515.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [11]:
@flow(name='Save from web to Cloud Storage')
def load_to_gcs(color: str, year: int, month: int) -> None:

    url = f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/{color}/{color}_tripdata_{year}-{month:02}.csv.gz'
    storage_path = f'trips/raw/{color}_tripdata_{year}-{month:02}.csv.gz'
    storage_path_parquet = f'trips/parquet/{color}_tripdata_{year}-{month:02}.parquet'

    content = extract_from_web(url)
    save_as_raw(content, storage_path)
    save_as_parquet(content, url, storage_path_parquet)

D:\DEZC\Week2\zoomcamp\lib\site-packages\prefect\flows.py:206: UserWarning: A flow named 'Save from web to Cloud Storage' and defined at 'C:\Users\ksale\AppData\Local\Temp\ipykernel_4392\2271283667.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [12]:
if __name__ == '__main__':
    color = 'green'
    year = 2020
    month = 1
    
    load_to_gcs(color, year, month)

01:21:59.759 | INFO    | prefect.engine - Created flow run 'translucent-pronghorn' for flow 'Save from web to Cloud Storage'

01:21:59.978 | INFO    | Flow run 'translucent-pronghorn' - Created task run 'extract_from_web-d388bed7-0' for task 'extract_from_web'

01:21:59.980 | INFO    | Flow run 'translucent-pronghorn' - Executing 'extract_from_web-d388bed7-0' immediately...

01:22:02.517 | INFO    | Task run 'extract_from_web-d388bed7-0' - Finished in state Completed()

01:22:02.571 | INFO    | Flow run 'translucent-pronghorn' - Created task run 'save_as_raw-fe301552-0' for task 'save_as_raw'

...

01:22:09.707 | INFO    | Task run 'save_as_parquet-dae70288-0' - Rows loaded: 447770

01:22:10.373 | INFO    | Task run 'save_as_parquet-dae70288-0' - Getting bucket 'prefect_zoomcamp'.

01:22:11.480 | INFO    | Task run 'save_as_parquet-dae70288-0' - Uploading from file object to the bucket 'prefect_zoomcamp' path 'trips\\parquet\\green_tripdata_2020-01.parquet'.

01:22:15.996 | INFO    | Task run 'save_as_parquet-dae70288-0' - Finished in state Completed()

01:22:16.045 | INFO    | Flow run 'translucent-pronghorn' - Finished in state Completed('All states completed.')